In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
import re # regular expression

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary # adds chromedriver binary to path
from selenium.webdriver.common.by import By # for wait time
from selenium.webdriver.support.ui import WebDriverWait # for wait time
from selenium.webdriver.support import expected_conditions as EC # for wait time conditions

In [32]:
def parsing_pages(list_of_pages):
    pages = []
    for page in list_of_pages:
        if pd.notnull(page):
            all_pages = requests.get(page)
            each_page = BeautifulSoup(all_pages.content, "html.parser")
            pages.append(each_page)
            sleep(randint(2,6))
        else: 
            pages.append(None)
    return pages

## Scarpping H Mart Grocery Items

In [33]:
hmart = ['https://www.hmart.com/groceries?p=' + str(i) for i in range(1,16)]
hmart_parsed = parsing_pages(hmart)

In [4]:
# finding item names from a list of beautifulsoup objects
key = 'product name product-item-name'
items = [] 
for page in hmart_parsed:
    items_in_pages = page.find_all('strong', class_=key)
    for item in items_in_pages:
        items_per_page = item.get_text().replace('\n','').replace('\r','').strip().replace('     ', ' ').lower()
        items.append(items_per_page)
# a = hmart_parsed[0].find_all('strong', class_=key)
# [i.get_text() for i in a]
brands = [i.split('        ')[0] for i in items] # 8 whitespaces for the best result
item_names = [i.split('        ')[-1] for i in items]

In [5]:
regrex = re.compile(r'\d\..*')
item_volumns = [regrex.findall(i) for i in item_names]
# item_test = []
# item_volumns = []
# for item in item_names:
#     item_volumn = regrex.findall(item)
#     item_test.append(item_volumn)
#     for i in item_volumn:
#         if i != '':
#             item_volumns.append(i)
#         else:
#             item_volumns.append(None)
#             print('No match!')

In [ ]:
## asian items with brand names
asian_items = []
for i, j in zip(brands, item_names):
    asian_item = i + ' ' + j
    asian_items.append(asian_item)

asian_items_no_volumns = []
for i in asian_items:
    asian_item = re.sub(' \d\..*', '', i)
    asian_items_no_volumns.append(asian_item)

# asian_items_no_volumns
# item_names

In [6]:
len(item_volumns)

704

In [244]:
columns = {
#     'brand_name': brands,
    'item_name': asian_items_no_volumns,
    'item_volumn': item_volumns,
    'type': 'asian'
}
asian_items = pd.DataFrame(data = columns)

In [71]:
asian_items.to_markdown.head()

NameError: name 'asian_items' is not defined

In [323]:
asian_items.to_csv('hmart.csv')

## Scraping Items from Other Supermarket

In [261]:
# walmart = ['https://www.walmart.com/browse/food/976759?page=' + str(i) for i in range(1,26)]
# walmart_parsed = parsing_pages(walmart)

In [270]:
# test = walmart_parsed[0].find_all('div', class_='search-result-product-title gridview')
# [i.find('a').get_text() for i in test] # dynamic websites

['Cheerios, Whole Grain Oats Cereal, Gluten Free, 18 oz',
 '(3 Pack) Lipton Beefy Onion Soup and Dip Mix, 2.2 oz',
 '(12 Count) Gatorade Thirst Quencher Sports Drink, Cool Blue, 12 fl oz',
 '(8 Cans) StarKist Chunk Light Tuna in Water, 5 oz',
 'Premier Protein Shake, Chocolate, 30g Protein, 11 Fl Oz, 4 Ct',
 'eat2explore Educational Food & Culture Explorer Box - Explore China!',
 'My College Crate Microwaveables Ultimate Snack Care Package for College Students - Variety Assortment of Microwaveables, Mac & Cheese, Popcorn, Ramen, Chips, Gummies & Candies (50 Snacks)',
 '(3 Pack) Betty Crocker Hamburger Helper, Double Cheeseburger Macaroni Hamburger Helper, 12.1 Oz Box (Twin Pack)',
 'Doritos Nacho Cheese Flavored Tortilla Chips, Party Size, 15 oz',
 'Kraft Original Flavor Mac and Cheese, 5 ct - 7.25 oz Multipack']

In [10]:
# kroger = ['https://www.kroger.com/pl/packaged-meals-sides/01011?page=' + 
#                     str(i) for i in range(1,13)]
# kroger_test = 'https://www.kroger.com/pl/packaged-meals-sides/01011?page=1'
# test = requests.get(kroger_test)
# test_parsed = BeautifulSoup(test.content, 'html.parser')

# test
# kroger_beverages_parsed = parsing_pages(kroger_beverages)

In [187]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary # adds chromedriver binary to path
from selenium.webdriver.common.by import By # for wait time
from selenium.webdriver.support.ui import WebDriverWait # for wait time
from selenium.webdriver.support import expected_conditions as EC # for wait time conditions

driver = webdriver.Chrome()
driver.get("https://www.walmart.com/browse/food/")

In [155]:
items = driver.find_elements_by_css_selector(".product-title-link.line-clamp span")

In [156]:
list_of_items = [i.text for i in items]

In [179]:
# driver.find_element_by_css_selector(".elc-icon.paginator-hairline-btn").click()

In [188]:
import time
from random import randint

list_of_items_from_pages = []
for n in range(1,26):
        driver.get("https://www.walmart.com/browse/food/?page=" + str(n))
        items_from_pages = driver.find_elements_by_css_selector(".product-title-link.line-clamp span")
        for i in items_from_pages:
            items_extracted = i.text
            list_of_items_from_pages.append(items_extracted)
        time.sleep(3)

In [189]:
len(list_of_items_from_pages)

994

In [203]:
walmart = pd.Series(list_of_items_from_pages)
walmart.to_csv('walmart.csv')

In [56]:
## re-openning the file, just gonna import previously scrapped csv files

walmart = pd.read_csv('walmart.csv')
hmart = pd.read_csv('hmart.csv')
walmart = walmart.iloc[:,1]

pack = re.compile(r'\(\d{1,} (?:pack|count|cans)\)') 
walmart = [re.sub(pack, '', i.lower()) for i in walmart]
walmart
regrex = re.compile(r'(?:, \d{1,}|\d{1,} (?:mg|oz|fl|ct)|, \d{1,} (?:mg|oz|fl|ct)).*')
walmart_item_volumns = [regrex.findall(i) for i in walmart]
walmart_item_names = [re.sub(regrex, '', i.split(',')[0].strip()) for i in walmart]



In [74]:
hmart.head().to_markdown()

"|    |   Unnamed: 0 | item_name                                | item_volumn              | type   |\n|---:|-------------:|:-----------------------------------------|:-------------------------|:-------|\n|  0 |            0 | cj hetbahn cooked white rice box         | ['7.4oz(210g) 12 ea']    | asian  |\n|  1 |            1 | cj cooked white rice with seaweed soup   | ['5.9oz(167g)']          | asian  |\n|  2 |            2 | samyang hot chicken flavor ramen         | ['4.94oz(140g) 5 packs'] | asian  |\n|  3 |            3 | cj cooked white rice with soft tofu stew | ['6.1oz(174g)']          | asian  |\n|  4 |            4 | haioreum peeled roasted chestnuts        | ['3.52oz(100g)']         | asian  |"

### Cleaning the Item Names

In [192]:
# getting rid of units/volumns at the beginning of each item
pack = re.compile(r'\(\d{1,} (?:pack|count|cans)\)') 
# list_of_items = [i.loewr() for i in list_of_items]
list_of_items_from_pages = [re.sub(pack, '', i.lower()) for i in list_of_items_from_pages]

# isolating volumns at the end
regrex = re.compile(r'(?:, \d{1,}|\d{1,} (?:mg|oz|fl|ct)|, \d{1,} (?:mg|oz|fl|ct)).*')
walmart_item_volumns = [regrex.findall(i) for i in list_of_items_from_pages]
walmart_item_names = [re.sub(regrex, '', i.split(',')[0].strip()) for i in list_of_items_from_pages]

In [57]:
columns = {
    'item_name': walmart_item_names,
    'item_volumn': walmart_item_volumns,
    'type': 'non-asian'
}
walmart_items = pd.DataFrame(data = columns)
walmart_items.head()


,item_name,item_volumn,type
0,lipton recipe secrets soup and dip mix onion,[2 oz 2 count],non-asian
1,doritos nacho cheese flavored tortilla chips,"[, 15 oz]",non-asian
2,equate 70% isopropyl alcohol,"[, 32 oz]",non-asian
3,purina dog chow complete with real chicken adu...,[],non-asian
4,assurance premium xl disposable washcloths,"[, 144 ct]",non-asian


## Tokenizing Item Names

In [62]:
## First, concat the two lists together

frames = [walmart_items, hmart]
df = pd.concat(frames).reset_index(drop = True)
# df.tail()

# seperate training and test sets
from random import sample
df_train = df.sample(frac=0.6, random_state=0)
df_test = df.drop(df_train.index)

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(stop_words='english',max_df=0.85)
train_vect = count_vect.fit_transform(df_train.item_name)

In [64]:
train_tf_transformer = TfidfTransformer(use_idf=False).fit(train_vect)
train_tf = train_tf_transformer.transform(train_vect)
train_tf.shape

(1019, 1719)

In [65]:
from sklearn.decomposition import NMF

nmf = NMF(n_components = 20, random_state=3,
          init='nndsvd')# .fit(upc_tf) # nndsvd for sparse data
#          alpha=.1, l1_ratio=.5)
# nmf_fit = nmf.fit(train_tf)

In [66]:
test_vect = count_vect.transform(df_test.item_name) # not fitting, but transforming
# test_tf_transformer = TfidfTransformer(use_idf=False).fit(train_vect)
test_tf = train_tf_transformer.transform(test_vect)
test_tf.shape

# test_nmf = nmf_fit.transform(test_tf)

(679, 1719)

In [67]:
from sklearn.linear_model import LogisticRegression

# features = 

X_train = train_tf
y_train = df_train.type

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

X_test = test_tf
y_test = df_test.type

log_reg.score(X_test, y_test)

0.946980854197349

## Test with Items at Kroger

In [291]:
driver = webdriver.Chrome()
driver.get("https://www.kroger.com/search?fulfillment=all&page=1&query=food")

In [278]:
test = driver.find_elements_by_css_selector(".kds-Text--m.text-default-800.mt-12.mb-4.font-500")
[i.text for i in test]

In [281]:
# [i.text for i in elements]

In [292]:
# from random import randint

kroger_items = []
for n in range(1,3):
    driver.get("https://www.kroger.com/search?fulfillment=all&page=" + str(n) + "&query=food")
    items = driver.find_elements_by_css_selector(".kds-Text--m.text-default-800.mt-12.mb-4.font-500")
    for i in items:
        items_extracted = i.text
        kroger_items.append(items_extracted)
        time.sleep(randint(3,10))

In [290]:
kroger_items

[]

(Kroger denied access :P)

## Test with Wegmans

In [313]:
# driver = webdriver.Chrome()
# driver.get("https://shop.wegmans.com/shop/categories/283?page=1")
# time.sleep(3)
# driver.find_element_by_css_selector("#shopping-selector-shop-context-intent-instore").click()
# time.sleep(3)
# text = driver.find_elements_by_css_selector(".cell-title")


In [320]:
driver = webdriver.Chrome()

wegmans_items = []
driver.get("https://shop.wegmans.com/shop/categories/283?page=1")
time.sleep(3)
driver.find_element_by_css_selector("#shopping-selector-shop-context-intent-instore").click()
time.sleep(3)
for n in range(1,133):
    time.sleep(2)
    driver.get("https://shop.wegmans.com/shop/categories/283?page=" + str(n))
    time.sleep(3)
    items = driver.find_elements_by_css_selector(".cell-title")
    for i in items:
        items_extracted = i.text
        wegmans_items.append(items_extracted)
    time.sleep(randint(3,10))

In [321]:
len(wegmans_items)

6691

In [322]:
wegmans_items = pd.Series(wegmans_items)
wegmans_items.to_csv('wegmans.csv')

In [316]:
testing = [i.text for i in text]
testing[0].split('\n')

['Wegmans Black Beans', '15.5 ounce', '15B']

In [329]:
wegmans_item_names = [i.split('\n')[0] for i in wegmans_items]
wegmans_item_volumns = [i.split('\n')[1] for i in wegmans_items]

In [330]:
columns = {
    'item_name': wegmans_item_names,
    'item_volumn': wegmans_item_volumns
}
wegmans = pd.DataFrame(data = columns)

In [331]:
wegmans_vect = count_vect.transform(wegmans.item_name)
wegmans_tf = train_tf_transformer.transform(wegmans_vect)
# wegmans_nmf = nmf_fit.transform(wegmans_tf)

In [334]:
X = wegmans_tf

y = log_reg.predict(X)

In [339]:
len(y[y == 'asian']) / len(y)

0.08369451502017636

In [349]:
wegmans['type'] = y

In [351]:
wegmans[wegmans['type'] == 'asian'].head(20)

,item_name,item_volumn,type
27,"Nissin Top Ramen Soup, Ramen Noodle, Chicken F...",18 ounce,asian
55,Wegmans Beef Culinary Stock,32 fl. oz.,asian
65,Rao's Homemade Marinara Sauce,24 ounce,asian
75,Wegmans Spicy Brown Mustard,12 ounce,asian
87,Wegmans Organic White Jasmine Rice,32 ounce,asian
120,Wegmans Organic Tomato Paste,6 ounce,asian
123,Wegmans Organic White Basmati Rice,32 ounce,asian
129,"Wegmans Whole Kernel Crisp'N Sweet Corn, No Sa...",15.25 ounce,asian
144,"Arrowhead Mills Flax Seeds, Organic",16 ounce,asian
145,"Wholesome Sugar, Organic, Dark Brown",24 ounce,asian


### Test with Wegmans' 'International Food' Collection

In [354]:
driver = webdriver.Chrome()

wegmans_international_items = []
driver.get("https://shop.wegmans.com/shop/categories/345?page=1")
time.sleep(3)
driver.find_element_by_css_selector("#shopping-selector-shop-context-intent-instore").click()
time.sleep(3)
for n in range(1,26):
    time.sleep(2)
    driver.get("https://shop.wegmans.com/shop/categories/345?page=" + str(n))
    time.sleep(3)
    items = driver.find_elements_by_css_selector(".cell-title")
    for i in items:
        items_extracted = i.text
        wegmans_international_items.append(items_extracted)
    time.sleep(randint(3,10))

In [355]:
wegmans_international_items_pd = pd.Series(wegmans_international_items)
wegmans_international_items_pd.to_csv('wegmans_international.csv')

In [356]:
wegmans_intl_item_names = [i.split('\n')[0] for i in wegmans_international_items]
wegmans_intl_item_volumns = [i.split('\n')[1] for i in wegmans_international_items]

In [357]:
columns = {
    'item_name': wegmans_intl_item_names,
    'item_volumn': wegmans_intl_item_volumns
}
wegmans_intl = pd.DataFrame(data = columns)

In [358]:
wegmans_intl_vect = count_vect.transform(wegmans_intl.item_name)
wegmans_intl_tf = train_tf_transformer.transform(wegmans_intl_vect)

In [359]:
X_intl = wegmans_intl_tf

y_intl = log_reg.predict(X_intl)

In [360]:
len(y_intl[y_intl == 'asian']) / len(y_intl)

0.30980945659844744

In [361]:
wegmans_intl['type'] = y_intl

In [362]:
wegmans_intl[wegmans_intl['type'] == 'asian']

,item_name,item_volumn,type
5,Wegmans Organic Tahini,16 ounce,asian
7,"Coconut Secret Seasoning Sauce, Soy-Free, Coco...",8 fl. oz.,asian
8,"Wegmans Organic Salsa, Hot",15.5 ounce,asian
11,Wegmans Organic General Tso's Sauce,14.5 ounce,asian
13,Wegmans Organic Coconut Cream,5.4 fl. oz.,asian
...,...,...,...
1396,"Kikkoman Soy Sauce, Smooth Aromatic, Shoyu",6.8 fl. oz.,asian
1405,"Nongshim Chajang Noodle, Chapagetti",4.5 ounce,asian
1410,"Paldo Soup, Bowl Noodle, Hot & Spicy Flavor",3.03 ounce,asian
1412,"Nongshim Udon Type Noodles, Spicy Seafood Flavor",4.2 ounce,asian


## Test with Family Dollar
(scrolling is too hard la)

In [370]:
# driver = webdriver.Chrome()

# familydollar_items = []
# driver.get("https://www.familydollar.com/categories?N=549612734&No=0&Nr=product.active:1")
# time.sleep(3)
# elements = driver.find_elements_by_css_selector(".product-title")
# for i in elements:
#     familydollar_items.append(i.text)
# # for n in range(1,26):
# #     time.sleep(2)
# #     driver.get("https://shop.wegmans.com/shop/categories/345?page=" + str(n))
# #     time.sleep(3)
# #     items = driver.find_elements_by_css_selector(".cell-title")
# #     for i in items:
# #         items_extracted = i.text
# #         wegmans_international_items.append(items_extracted)
# #     time.sleep(randint(3,10))

In [372]:
# for i in elements:
#     familydollar_items.append(i.text)
# len(familydollar_items)

29

In [373]:
# driver = webdriver.Chrome()

# familydollar_items = []
# driver.get("https://www.familydollar.com/categories?N=549612734&No=0&Nr=product.active:1")
# time.sleep(3)
# item_wrapper = driver.find_elements_by_css_selector(".product-wrapper.items-per-row-3")

# # get_number = 0
# # while True:
# #     count = get_number
# #     driver.execute_script("arguments[0].scrollIntoView();", )

    

In [391]:
# item_wrapper = driver.find_elements_by_css_selector(".product-wrapper.items-per-row-3.items-per-row-mobile-2")

In [392]:
# test = []
# for i in item_wrapper:
#     testing = i.find_elements_by_css_selector(".search-product-link")
#     test.append(testing)

# # test = item_wrapper.find_elements_by_css_selector(".search-product-link")

## Testing with Harris Teeter

In [10]:
# driver = webdriver.Chrome()

harris_teeter_canned = []
driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,1557/products?isSpecialSubCategory=false")
sleep(3)
elements = driver.find_elements_by_css_selector(".product-name")
for i in elements:
    harris_teeter_canned.append(i.text)
# for n in range(1,26):
#     time.sleep(2)
#     driver.get("https://shop.wegmans.com/shop/categories/345?page=" + str(n))
#     time.sleep(3)
#     items = driver.find_elements_by_css_selector(".cell-title")
#     for i in items:
#         items_extracted = i.text
#         wegmans_international_items.append(items_extracted)
#     time.sleep(randint(3,10))

In [12]:
harris_teeter_noodles = []
driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,976/products?isSpecialSubCategory=false")
sleep(3)
elements = driver.find_elements_by_css_selector(".product-name")
for i in elements:
    harris_teeter_noodles.append(i.text)



In [14]:
harris_teeter_noodles_2 = []
driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,976/products?pageNo=2&appliedSort=Brand")
sleep(3)
elements = driver.find_elements_by_css_selector(".product-name")
for i in elements:
    harris_teeter_noodles_2.append(i.text)

In [16]:
harris_teeter_noodles = harris_teeter_noodles + harris_teeter_noodles_2

In [18]:
harris_teeter_other = []
driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,1558/products?isSpecialSubCategory=false")
sleep(3)
elements = driver.find_elements_by_css_selector(".product-name")
for i in elements:
    harris_teeter_other.append(i.text)

In [19]:
harris_teeter_other

['Annie Chuns Soup Bowl, Coconut, Thai-Style, Medium',
 'Hapi Wasabi Peas - Hot',
 'HT Traders Wasabi Peas',
 'Pocky Biscuit Sticks Covered with Strawberry Cream',
 'Shirakiku Ramune Drink, Carbonated, Strawberry',
 'Sushi Chef Pickled Ginger',
 'Thai Kitchen Aseptic Coconut Milk']

In [20]:
harris_teeter_sauce = []
driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,912/products?isSpecialSubCategory=false")
sleep(3)
elements = driver.find_elements_by_css_selector(".product-name")
for i in elements:
    harris_teeter_sauce.append(i.text)

In [21]:
harris_teeter_sauce = []
for n in range(1,4):
    driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,912/products?pageNo=" + str(n) +  "&appliedSort=Brand")
    sleep(randint(2,6))
    elements = driver.find_elements_by_css_selector(".product-name")
    for i in elements:
        harris_teeter_sauce.append(i.text)
    sleep(randint(2,6))


In [23]:
harris_teeter_seasoning = []
driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,1231/products?isSpecialSubCategory=false")
sleep(3)
elements = driver.find_elements_by_css_selector(".product-name")
for i in elements:
    harris_teeter_seasoning.append(i.text)

In [26]:
harris_teeter_ramen = []
for n in range(1,3):
    driver.get("https://www.harristeeter.com/shop/store/332/category/583/subCategory/713,1164/products?pageNo=" + str(n) + "&appliedSort=Brand")
    sleep(3)
    elements = driver.find_elements_by_css_selector(".product-name")
    for i in elements:
        harris_teeter_ramen.append(i.text)

In [28]:
harris_teeter_asian = harris_teeter_ramen + harris_teeter_seasoning + harris_teeter_sauce + harris_teeter_other + harris_teeter_noodles + harris_teeter_canned

In [30]:
harris_teeter_asian = pd.Series(harris_teeter_asian)
harris_teeter_asian.to_csv("harris_teeter_asian.csv")

In [68]:
ht_asian_vect = count_vect.transform(harris_teeter_asian)
ht_asian_tf = train_tf_transformer.transform(ht_asian_vect)

In [69]:
X_ht = ht_asian_tf

y_ht = log_reg.predict(X_ht)

In [70]:
len(y_ht[y_ht == 'asian']) / len(y_ht)

0.5808823529411765